## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv(r'C:/UCD/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv')
# city_data_df = pd.read_scv(r'C:/Users/s.etoumi/Desktop/UCD/World_Weather_Analysis/Weather_Database.csv')
city_data_df = pd.read_csv('/Users/s.etoumi/Desktop/UCD/World_Weather_Analysis/WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jamestown,42.0970,-79.2353,53.24,64,100,13.80,US,2022-11-12 07:39:09
1,1,Saldanha,-33.0117,17.9442,69.85,71,0,16.06,ZA,2022-11-12 07:42:43
2,2,Mataura,-46.1927,168.8643,50.68,66,100,1.90,NZ,2022-11-12 07:42:43
3,3,Provideniya,64.3833,-173.3000,14.92,75,2,9.71,RU,2022-11-12 07:42:44
4,4,Kirovsk,48.6375,38.6428,44.20,81,100,9.26,UA,2022-11-12 07:42:45


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Vaini,-21.2000,-175.2000,78.67,85,13,5.37,TO,2022-11-12 07:42:46
9,9,Port Elizabeth,-33.9180,25.5701,79.09,57,0,8.05,ZA,2022-11-12 07:41:09
11,11,Shanghai,31.2222,121.4581,80.46,76,0,8.95,CN,2022-11-12 07:40:24
19,19,Padang,-0.9492,100.3543,80.65,83,100,3.69,ID,2022-11-12 07:42:52
28,28,Bredasdorp,-34.5322,20.0403,75.31,42,0,8.61,ZA,2022-11-12 07:42:57
31,31,Hurghada,27.2574,33.8129,75.45,52,0,15.39,EG,2022-11-12 07:42:58
35,35,Hobyo,5.3505,48.5268,85.51,57,7,16.62,SO,2022-11-12 07:43:00
41,41,Butaritari,3.0707,172.7902,81.30,75,93,13.24,KI,2022-11-12 07:43:03
53,53,Mkokotoni,-5.8767,39.2547,84.36,63,25,2.46,TZ,2022-11-12 07:44:09
58,58,Port Hedland,-20.3167,118.5667,86.05,48,0,18.41,AU,2022-11-12 07:44:11


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       198
City          198
Lat           198
Lng           198
Max Temp      198
Humidity      198
Cloudiness    198
Wind Speed    198
Country       197
Date          198
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()
#preferred_cities_df.count()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Vaini,-21.2000,-175.2000,78.67,85,13,5.37,TO,2022-11-12 07:42:46
9,9,Port Elizabeth,-33.9180,25.5701,79.09,57,0,8.05,ZA,2022-11-12 07:41:09
11,11,Shanghai,31.2222,121.4581,80.46,76,0,8.95,CN,2022-11-12 07:40:24
19,19,Padang,-0.9492,100.3543,80.65,83,100,3.69,ID,2022-11-12 07:42:52
28,28,Bredasdorp,-34.5322,20.0403,75.31,42,0,8.61,ZA,2022-11-12 07:42:57


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

KeyError: "['Current Description'] not in index"

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
